## Import Library and Acquire Work Directory

In [ ]:
# system library
import os
import glob
import shutil
from importlib import reload

# 3rd party library
import numpy as np
import pandas as pd
import lmfit
import KiMoPack.plot_func as pf
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

# user library
import min_function as mf
import min_instrument as mins
import min_math as mm
import min_plot as mp

# acquire current work directory
cwd = os.getcwd()
print(f"\nCurrent Work Directory:\n{cwd}")

# acquire filename from current work directory
fn = os.path.basename(cwd)
fn = fn.split("_")
fn = "_".join(fn[0:2])
print(f"\nFilename:\n{fn}")

# remove the folders
# shutil.rmtree("Scans")
# shutil.rmtree("result_figures")
# shutil.rmtree("p2")
# shutil.rmtree("g2")
# shutil.rmtree("t2")

# Preprocess

In [ ]:
# acquire the filepath of each scan
fps_scan = glob.glob(f"{cwd}/*/*.GUI.csv")
fps_scan.sort()
print(f"\n{len(fps_scan)} scans were found:")
for fp_scan in fps_scan:
    print(os.path.basename(fp_scan))

# create the Scans folder for storing the processed data
os.makedirs(f"{cwd}/Scans", exist_ok=True)

## Process Each Scan

In [ ]:
# create the ls_scan list for storing the processed dataframe
ls_scan = []
# fps_scan = fps_scan[0:1]  # for testing, only process the first scan
for fp_scan in fps_scan:
    print(f"\nCurrent scan:\n{os.path.basename(fp_scan)}")

    # modify the filename
    (dir_scan, fn_scan) = os.path.split(fp_scan)
    fn_scan = os.path.basename(fp_scan)
    fn_scan = fn_scan.split(".")
    scan_count = ((fn_scan[1].split("_"))[-1]).lower()
    fn_scan = fn_scan[0] + "_" + scan_count
    # fn_scan = fn_scan.replace("uvpr", "caf2")  # adjust this line for proper filename
    print(f"\nModified Filename:\n{fn_scan}")

    # import raw data in mOD
    scan_df = pd.read_csv(fp_scan, header=0, index_col=0, na_values="NaN", sep=",")
    scan_df = scan_df * 1000  # convert OD to mOD
    scan_df.columns = [float(i) for i in scan_df.columns]
    print(
        f"\nRaw fs-TA data, \033[1m{len(np.where(scan_df.isna())[0])} NaN point\033[0m:"
    )
    # display(scan_df)
    mp.display_fsta_heatmap(scan_df).show()
    mins.writeSXUfs(
        scan_df,
        f"Scans/{fn_scan}_raw.ufs",
    )

    # crop the data to the wl_range
    wl_range = [330, 750]
    tailored_scan_df = mins.tailor_fsta(scan_df, wl_range)
    tailored_scan_df = scan_df.loc[
        (scan_df.index >= wl_range[0]) & (scan_df.index <= wl_range[1])
    ]
    print(
        f"\nTailored raw fs-TA data, \033[1m{len(np.where(tailored_scan_df.isna())[0])} NaN points\033[0m:"
    )
    # display(tailored_scan_df)
    # mp.display_fsta_heatmap(tailored_scan_df).show()
    mins.writeSXUfs(
        tailored_scan_df,
        f"Scans/{fn_scan}_{wl_range[0]}{wl_range[1]}.ufs",
    )

    # repair NaN points
    rpnan_tailored_scan_df = tailored_scan_df.interpolate(
        method="linear", limit_direction="both", axis=0
    )  # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.interpolate.html
    print(
        f"Repaired tailored raw fs-TA data, \033[1m{len(np.where(rpnan_tailored_scan_df.isna())[0])} NaN point\033[0m:"
    )
    # display(rpnan_tailored_scan_df)
    # mp.display_fsta_heatmap(rpnan_tailored_scan_df).show()
    mins.writeSXUfs(
        rpnan_tailored_scan_df,
        f"Scans/{fn_scan}_{wl_range[0]}{wl_range[1]}_rpnan.ufs",
    )

    # store each scan in ls_scan
    ls_scan.append(rpnan_tailored_scan_df)

## Compare Each Scan


In [ ]:
ref_scan_num = 0
for i in range(1, len(ls_scan)):
    scan_diff = mins.compare_scan_fsta(ls_scan[ref_scan_num], ls_scan[i])
    mp.display_fsta_heatmap(scan_diff, zlimit=[-2, 2, 0.1]).show()
    mins.writeSXUfs(
        scan_diff,
        f"Scans/{fn}_diff{i}{ref_scan_num}.ufs",
    )

## Reaverage Data

In [ ]:
average_fsta = mins.reaverage_fsta(ls_scan[:3])
# display(average_fsta)
mp.display_fsta_heatmap(average_fsta).show()
mins.writeSXUfs(
    average_fsta,
    f"Scans/{fn}_rna_rav.ufs",
)

## Subtract Background

In [ ]:
sub_bg_fsta = mins.subtract_background_fsta(average_fsta, [-100, -2])
# display(sub_bg_fsta)
mp.display_fsta_heatmap(sub_bg_fsta).show()
mins.writeSXUfs(
    sub_bg_fsta,
    f"Scans/{fn}_rna_rav_sbg.ufs",
)

# Visualize


In [ ]:
sample = f"Sample"  # legend name shown in figure
fp = glob.glob(f"{cwd}/*.ufs")[0]  # only one .ufs file in the current work directory
data = mins.load_fsta(fp)

data = data.iloc[:, 1:]  # remove the first column, usually contains NaN points
nan = np.where(data.isna())  # check if there is NaN points
display(nan)
display(data)

# acquire the maximum and minimum values for setting the mOD range
globalmax = data.max().max()
globalmin = data.min().min()
localmax = mins.tailor_fsta(data, [355, 700], [0.3, 8000]).max().max()
localmin = mins.tailor_fsta(data, [355, 700], [0.3, 8000]).min().min()
print(
    f"globalmax: {globalmax}\nglobalmin: {globalmin}\nlocalmax: {localmax}\nlocalmin: {localmin}"
)

## 2D Map


In [ ]:
# reload(mp)
# mp.display_fsta_heatmap_linear(data).show()

nm_limit = [[345, 650], 50, 10]  # set the wavelength range
e3cm1_limit = [
    [mf.e3cm1_to_nm(nm_limit[0][0]), mf.e3cm1_to_nm(nm_limit[0][1])],
    2,
    0.5,
]  # convert to 10**3 wavenumber
t_limit = [[-5, 100, 8000], 50, 10]  # set the time window
od_limit = [[-localmax, localmax], 2, 1]  # set the mOD range

contour_bwn_twl = mp.display_fsta_contourf_symlog_be3cm1_tnm(
    df_fsta=data,
    legendtitle=sample,
    xlimit=e3cm1_limit,
    xlimit2=nm_limit,
    ylimit=t_limit,
    zlimit=od_limit,
)
plt.show()

## Spectra


In [ ]:
# reload(mp)

delaytime1 = [0.445, 75.3, 200, 457, 1500]  # set the delay time 1
spectra1 = mins.extract_spectra_trspectra(data, delaytime1)
# display(spectra1)
fstas_bwn_twl = mp.display_fsta_spectra_be3cm1_tnm(
    df_fstas_wl=spectra1,
    legendtitle=sample,
    xlimit=e3cm1_limit,
)
plt.show()

delaytime2 = [1710, 3490, 7370]  # set the delay time 2
spectra2 = mins.extract_spectra_trspectra(data, delaytime2)
# display(spectra2)
fstas_bwn_twl2 = mp.display_fsta_spectra_be3cm1_tnm(
    df_fstas_wl=spectra2,
    legendtitle=sample,
    xlimit=e3cm1_limit,
)
plt.show()

## Kinetics


In [ ]:
# reload(mp)
kinetics_t_limit = [[-100, 2000, 8000], 500, 100]
kinetics_od_limit = [[-1, 12], 2, 1]

wavelengths = [357, 472]  # set the wavelengths for extracting kinetics
kinetics = mins.extract_2colkinetics_trspectra(data, wavelengths)
# display(kinetics)
# mp.preview_2coldf(kinetics)
mp.display_fsta_2colkinetics_symlog(
    kinetics,
    legendtitle=sample,
    xlimit=kinetics_t_limit,
    ylimit=kinetics_od_limit,
    legendposition="upper left",
)
plt.show()

#  Single Wavelength Fitting


In [ ]:
func = "GPdfConHDExp"  # fitting function
fits = []

# set a loop for fitting kinetics at each wavelength
# be aware that the initial parameters and bounds might not suit each wavelength
for i in range(int(kinetics.shape[1] / 2)):
    fit = mm.fit_fsta_2colkinetics(
        data=kinetics.iloc[:, [2 * i, 2 * i + 1]],
        func_name=func,
        p0=[0, 0, 0.2, 5, 300, 3, 50000],
        # bounds=(
        #     [-0.00001, -2, -1, 0, 295, 0, 0],
        #     [0.00001, 2, 1, np.inf, np.inf, np.inf, np.inf],
        # ),
        yaxistype="linear",
    )
    fits.append(fit[0])

# if the loop fitting doesn't work, try fitting each wavelength separately
# fit1 = mm.fit_fsta_2colkinetics(
#     data=kinetics.iloc[:, 0:2],
#     func_name=func,
#     p0=[0, 0, 0.2, 5, 300, 3, 50000],
#     bounds=(
#         [-0.00001, -2, -1, 0, 295, 0, 0],
#         [0.00001, 2, 1, np.inf, np.inf, np.inf, np.inf],
#     ),
#     yaxistype="linear",
# )
# fits.append(fit1[0])

# fit2 = mm.fit_fsta_2colkinetics(
#     data=kinetics.iloc[:, 2:],
#     func_name=func,
#     p0=[0, 0, 0.2, -3, 300, 12, 50000],
#     bounds=(
#         [-0.00001, -1, 0, -np.inf, 0, 0, 40000],
#         [0.00001, 1, 1, 0, np.inf, np.inf, np.inf],
#     ),
#     yaxistype="linear",
# )
# fits.append(fit2[0])

In [ ]:
mp.display_fsta_1colkinetics_fitted_symlog(
    fits,
    xlimit=kinetics_t_limit,
    ylimit=kinetics_od_limit,
    legendtitle=sample,
    legendposition="upper left",
    # showwn=True,
)
plt.show()

# Global Analysis


In [ ]:
# plt.close("all")

ta = pf.TA(filename=sample, ds=data.T, sep=",", transpose=False)
# ta.lintresh = 100

# spectra
ta.wave_nm_bin = 1
ta.rel_time = delaytime1 + delaytime2
# ta.time_width_percent=0
# ta.timelimits = [ylimit[0][0], ylimit[0][2]]
ta.timelimits = [-100, 8000]
# ta.ignore_time_region=[-0.5, 0.5]
ta.bordercut = nm_limit[0]  # wavelength range

# kinetics
ta.wavelength_bin = 1
ta.rel_wave = wavelengths
# ta.scattercut=[300, 325]

# DA
ta.cmap = pf.cm.jet
ta.intensity_range = od_limit[0]
# ta.log_scale=False
# display(ta.ds)

# fit
ta.error_matrix_amplification = 10
ta.log_fit = False

# plot
# ta.log_scale=False
ta.Plot_RAW(
    plotting=[3],  # only ouput the result of SVD analysis
    # scale_type="symlog"
)
# ta.Plot_Interactive()

## Parallel Model


In [ ]:
ta.mod = "paral"  # or "exponential"
path = "p2"

ta.par = lmfit.Parameters()
ta.par.add("k0", value=1 / 300, vary=True)
ta.par.add("k1", value=1 / 150000, vary=True)
ta.par.add("t0", value=0, min=-1, max=1, vary=True)
ta.par.add("resolution", value=0.2, min=0, max=1, vary=False)
# ta.par.add("infinite")
# ta.par.add("background")

ta.Fit_Global(fit_chirp=False)
ta.Plot_fit_output(path=path)

ta.Save_data(
    save_RAW=False,
    path=path,
    filename=fn.replace(".csv", "") + "_" + path,
    sep=",",
)
# ta.Save_Plots(
#              path=os.getcwd()+"/parallelST",
#              savetype="pdf",
#              filename=filename.replace(".csv", "") + "_parallelST",
# )

## Sequential Model


In [ ]:
reload(mp)

ta.mod = "consecutive"
path = "g2"
ta.par = lmfit.Parameters()
ta.par.add("k0", value=1 / 300, vary=True)
ta.par.add("k1", value=1 / 10000000, vary=True)
ta.par.add("t0", value=0, min=-1, max=1, vary=True)
ta.par.add("resolution", value=0.25, min=0, max=1, vary=False)
# ta.par.add("infinite")
# ta.par.add("background")

ta.Fit_Global(fit_chirp=False, sub_sample=20)
ta.Plot_fit_output(path=path)

ta.Save_data(
    save_RAW=False,
    path=path,
    filename=fn.replace(".csv", "") + "_" + path,
    sep=",",
)

In [ ]:
# reload(mp)

df_raw_kinetics = glob.glob("g2/*_measured_kinetics.dat")
# display(df_raw_kinetics)
df_raw_kinetics = pd.read_csv(df_raw_kinetics[0], header=0, index_col=0, sep=",")
# display(df_raw_kinetics)
# mp.preview_1col_df(df_raw_kinetics)

df_fit_kinetics = glob.glob("g2/*_fitted_kinetics.dat")
# display(df_fit_kinetics)
df_fit_kinetics = pd.read_csv(df_fit_kinetics[0], header=0, index_col=0, sep=",")
# display(df_fit_kinetics)
# mp.preview_1col_df(df_fit_kinetics)

list_df_raw_fit_kinetics = mf.combine_data_fit_kinetics(
    df_raw_kinetics, df_fit_kinetics
)
# display(list_df_raw_fit_kinetics)

mp.display_fsta_1colkinetics_fitted_symlog(
    list_df_raw_fit_kinetics[:2],
    xlimit=kinetics_t_limit,
    ylimit=kinetics_od_limit,
    legendposition="upper left",
    legendtitle=sample,
    # showwn=True,
)
plt.show()

In [ ]:
# reload(mp)

df_eads = ta.re["DAC"]
# mp.preview_1coldf(df_eads)
tau1 = mf.formalize_fsta_delaytime(ta.re["fit_results_times"].iloc[0, 0])
tau2 = mf.formalize_fsta_delaytime(ta.re["fit_results_times"].iloc[1, 0])
# display(tau1, tau2)

fig_eads = mp.display_fsta_globalfit_spectra_be3cm1_tnm(
    df_fstas_wl=df_eads,
    xlimit=e3cm1_limit,
    legendtitle=sample,
    legendtext=[
        f"A, k\u2081\u207b\u00b9 : {tau1}",
        f"B, k\u2082\u207b\u00b9 : {tau2}",
    ],
)
plt.show()

In [ ]:
# reload(mp)

df_c = ta.re["c"]
# mp.preview_1coldf(df_c)

fig_c = mp.display_fsta_globalfit_concentration_symlog(
    df_1col=df_c,
    xlimit=kinetics_t_limit,
    legendtitle=sample,
    legendtext=["A", "B"],
)
plt.show()

## Target Model


In [ ]:
ta.mod = mm.targetST
path = "t2"
# k0: s1 decay
# k1: ISC
# k2: t1 decay
ta.par = lmfit.Parameters()
ta.par.add("k0", value=1 / 4600, vary=True)
ta.par.add("k1", value=1 / 8, vary=True)
ta.par.add("k2", value=1 / 4600, vary=True)
ta.par.add("t0", value=0, min=-1, max=1, vary=True)
ta.par.add("resolution", value=0.20, min=0, max=1, vary=True)
ta.Fit_Global(fit_chirp=False, sub_sample=10)
ta.Plot_fit_output(path=path)

ta.Save_data(
    save_RAW=False,
    path=path,
    filename=fn.replace(".csv", "") + "_" + path,
    sep=",",
)